# Constitution du cloud MinIO Client 

Le fichier diffusion du cloud MinIO Client est composé :
- du fichier 'cds_data' qui regroupe les données issues du modèle de projection CMIP6 mis à disposition par le Climate Data Store de Copernicus 
- du fichier 'puffin_data' 

Ce notebook a pour objectif de centraliser les requêtes API réalisées pour obtenir les données climatiques issues du modèle prédictif CMIP6 et leur écriture sur le cloud ainsi que l'écriture des données relatives aux macaraux moines(Atlantic puffins en anglais).

## 0. Set up the work environment for implementing data on the MinIO Client cloud 

In [ ]:
import cdsapi
import pandas as pd
import s3fs
import zipfile
import xarray as xr
import dotenv
import os



# Définir les chemins dynamiques dans un sous-dossier "data/"
data_dir = "./data"  # Sous-dossier pour centraliser les données
os.makedirs(data_dir, exist_ok=True)  # Crée le dossier s'il n'existe pas
temp_dir = os.path.join(data_dir, "extracted_files") # Sous-dossier d'extraction des fichiers zip
os.makedirs(temp_dir, exist_ok=True)  # Crée le dossier d'extraction


# Required information for the connection to Eve's bucket on MinIO Client cloud
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"
print(fs.ls(MY_BUCKET))


# Using of key elements for the API which are written in a hidden file '.cdsapirc'
client = cdsapi.Client()

### Nota bene 

The 'cdsapi' package has been developed by the Copernicus Climate Data Store (CDS) to perform API requests. The API url and token have first been entered in a '.cdsapirc' file, enabling the 'cdsapi.Client()' command to enter the connection information needed to perform the API. This file has been added to the .gitignore file, as it contains connection information for the CDS platform. 

## 1. API - Climate Data Store

### API CDS n°1 
Modèle : hadgem3_gc31_ll (UK)
Données : historiques [1950 - 2014] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

In [29]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "historical",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "year": [
        "1950", "1951", "1952",
        "1953", "1954", "1955",
        "1956", "1957", "1958",
        "1959", "1960", "1961",
        "1962", "1963", "1964",
        "1965", "1966", "1967",
        "1968", "1969", "1970",
        "1971", "1972", "1973",
        "1974", "1975", "1976",
        "1977", "1978", "1979",
        "1980", "1981", "1982",
        "1983", "1984", "1985",
        "1986", "1987", "1988",
        "1989", "1990", "1991",
        "1992", "1993", "1994",
        "1995", "1996", "1997",
        "1998", "1999", "2000",
        "2001", "2002", "2003",
        "2004", "2005", "2006",
        "2007", "2008", "2009",
        "2010", "2011", "2012",
        "2013", "2014"
    ],
    "area": [63.5, 20.5, 63.3, 20.1]
}

client = cdsapi.Client()
final_zip_path = os.path.join(data_dir, "cds_output_vestmann.zip")
client.retrieve(dataset, request).download(final_zip_path)


# Extraction of the zipfile downloaded from the API
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction in extracted_files
print(f"Files extracted in : {temp_dir}")



# Writing of the extracted file on the MinIO Client cloud
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc'
file_path = "data/extracted_files/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc"
with fs.open(minio_path, "wb") as minio_file:  # Target file on MinIO
    with open(file_path, "rb") as local_file:  # Local original file 
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))


2024-12-23 17:09:41,286 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-23 17:09:41,287 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-12-23 17:09:41,289 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-12-23 17:09:41,290 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

Files extracted in : ./data/extracted_files
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc']


### API CDS n°2 - SSP 5-8.5 (scénario le plus pessimiste)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

In [30]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp5_8_5",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp5_8_5.zip")
client.retrieve(dataset, request).download(final_zip_path)



# Extraction of the zipfile downloaded from the API
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")



# Writing of the extracted file on the MinIO Client cloud
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc'
file_path = "data/extracted_files/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc"
with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))


2024-12-23 17:11:50,631 INFO Request ID is c3de168a-9cbd-422a-bbb7-5cc3ab394bb3
2024-12-23 17:11:50,995 INFO status has been updated to accepted
2024-12-23 17:11:56,203 INFO status has been updated to running
2024-12-23 17:12:12,397 INFO status has been updated to successful
                                                                                        

Fichiers extraits dans : ./data/extracted_files
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc']


### API CDS n°2 - SSP 1-2.6 (scénario le plus optimiste)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

In [31]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp1_2_6",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp1_2_6.zip")
client.retrieve(dataset, request).download(final_zip_path)


# Extraction of the zipfile downloaded from the API
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")



# Writing of the extracted file on the MinIO Client cloud
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc'
file_path = "data/extracted_files/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc"
with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

2024-12-23 17:13:37,215 INFO Request ID is c92f6f9e-2fc6-41a3-af07-270733dd7b29
2024-12-23 17:13:37,474 INFO status has been updated to accepted
2024-12-23 17:13:58,766 INFO status has been updated to successful


Fichiers extraits dans : ./data/extracted_files
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc']


### API CDS n°4 - SSP 2-4.5 (scénario intermédiaire)
Modèle : hadgem3_gc31_ll (UK)
Données : prédictives [2015 - 2049] 
Localisation : îles Vestmann (Islande)
Variable : Sea Surface Temperature

In [32]:
dataset = "projections-cmip6"
request = {
    "temporal_resolution": "monthly",
    "experiment": "ssp2_4_5",
    "variable": "sea_surface_temperature",
    "model": "hadgem3_gc31_ll",
    "year": [
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024", "2025", "2026",
        "2027", "2028", "2029",
        "2030", "2031", "2032",
        "2033", "2034", "2035",
        "2036", "2037", "2038",
        "2039", "2040", "2041",
        "2042", "2043", "2044",
        "2045", "2046", "2047",
        "2048", "2049"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "area": [63.5, 20.5, 63.3, 20.1],  # Zone : Nord, Ouest, Sud, Est
}

final_zip_path = os.path.join(data_dir, "cds_output_vestmann_ssp2_4_5.zip")
client.retrieve(dataset, request).download(final_zip_path)


# Extraction of the zipfile downloaded from the API
with zipfile.ZipFile(final_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)  # Extraction dans extracted_files
print(f"Fichiers extraits dans : {temp_dir}")



# Writing of the extracted file on the MinIO Client cloud
minio_path = f'{MY_BUCKET}/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc'
file_path = "data/extracted_files/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc"
with fs.open(minio_path, "wb") as minio_file:  # Fichier cible sur MinIO
    with open(file_path, "rb") as local_file:  # Fichier source local
        minio_file.write(local_file.read())

print(fs.ls(f"{MY_BUCKET}/diffusion/cds_data"))

2024-12-23 17:15:09,360 INFO Request ID is 0128e360-848f-4edf-84d4-848c950b8e74
2024-12-23 17:15:09,476 INFO status has been updated to accepted
2024-12-23 17:15:14,443 INFO status has been updated to running
2024-12-23 17:15:30,884 INFO status has been updated to successful
                                                                                        

Fichiers extraits dans : ./data/extracted_files
['esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc', 'esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc']


## 2. Données relatives aux Atlantic puffins (macareux moines)

### Conversion des feuilles de données Excel en fichiers CSV

In [ ]:
excel_file = "./data/puffin-datas1.xlsx"  # Remplacez par le chemin de votre fichier Excel
# 3. Charger toutes les feuilles du fichier Excel
xls = pd.ExcelFile(excel_file)

for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    csv_file = os.path.join(data_dir, f"puffin-data1_{sheet_name}.csv")
    
    # Sauvegarder la feuille en CSV
    df.to_csv(csv_file, index=False, encoding="utf-8")
    print(f"Feuille '{sheet_name}' enregistrée dans : {csv_file}")

Feuille 'Title page' enregistrée dans : ./data/puffin-data1_Title page.csv
Feuille 'Data' enregistrée dans : ./data/puffin-data1_Data.csv
Feuille 'Explanations (Data)' enregistrée dans : ./data/puffin-data1_Explanations (Data).csv
Feuille 'Colonies' enregistrée dans : ./data/puffin-data1_Colonies.csv
Feuille 'Explanations (Colonies)' enregistrée dans : ./data/puffin-data1_Explanations (Colonies).csv


### Ecriture des nouveaux fichiers CSV dans le cloud MinIO Client

In [6]:
local_folder = "./data"

for file_name in os.listdir(local_folder):
    if file_name.startswith("puffin-data1"):
        local_file_path = os.path.join(local_folder, file_name)
        minio_path = f"{MY_BUCKET}/diffusion/puffin_data/{file_name}"
        
        with open(local_file_path, "rb") as local_file:
            with fs.open(minio_path, "wb") as minio_file:
                minio_file.write(local_file.read())
        
        print(f"Uploaded {file_name} successfully!")



Uploaded puffin-data1_Title page.csv successfully!
Uploaded puffin-data1_Data.csv successfully!
Uploaded puffin-data1_Colonies.csv successfully!
Uploaded puffin-data1_Explanations (Data).csv successfully!
Uploaded puffin-data1_Explanations (Colonies).csv successfully!
